In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = 'https://www.goodreads.com/list/show/2681.Time_Magazine_s_All_Time_100_Novels'
response = requests.get(url)

soup = BeautifulSoup(response.content, 'html.parser')
table = soup.find('table', class_='tableList')

books = []

ranks = soup.findAll("td", class_="number")
titles = soup.findAll("a", class_="bookTitle")
authors = soup.findAll("a", class_="authorName")
ranks = [r.text for r in ranks]
titles = [r.text for r in titles]
authors = [r.text for r in authors]
df = pd.DataFrame({'Rank': ranks, 'Title': titles, 'author': authors})



In [2]:
# connent to openai
import openai
import shhh
openai.api_key = shhh.aikey


In [29]:


def chat_request(question):

    response = openai.Completion.create(
        model="text-davinci-003",
        prompt=question,
        temperature=0.8,
        max_tokens=2000,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
    )

    print(response)

    a = response.choices[0].text
    print(a)
    return a


In [31]:
# loop through df and present prompt to gpt
summaries = {}
for i, row in df.iterrows():
    if i == 1: break
    auth = row['author']
    title = row['Title'].replace("\n", "")
    # prompt = f"pretend to be {auth} and summarize the middle of {title} in less than 150 words"
    p_1 = f"Summarize the beginning of {title} by {auth}\n\n"
    p_2 = f"in the writing style of the author in <150 words\n\n"
    p_3 = f"here's an excerpt: Maycomb was a tired old town, even in 1932 when I first knew it. Somehow, it was hotter then. Mens stiff collars wilted by nine in the morning. Ladies bathed before noon, after their three-oclock naps, and by nightfall were like soft teacakes with frostings of sweat and sweet talcum. The day was twenty-four hours long, but it seemed longer. Theres no hurry, for theres nowhere to go and nothing to buy...\n"
    p_4 = f"Summary:"

    bs = chat_request(p_1+p_2+p_3+p_4)
    summaries[i] = bs

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\nMaycomb, a tired old town in 1932, was sweltering hot. Men wilted in their stiff collars and ladies would take refreshing baths in the mornings and become sweaty and fragrant by nightfall. The day seemed to last much longer than its 24 hours, for there was no need to hurry - there was nowhere to go and nothing to buy."
    }
  ],
  "created": 1682720179,
  "id": "cmpl-7AQfzbe6CqASNpk5G7A9B5dLMOdjV",
  "model": "text-davinci-003",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 74,
    "prompt_tokens": 139,
    "total_tokens": 213
  }
}

Maycomb, a tired old town in 1932, was sweltering hot. Men wilted in their stiff collars and ladies would take refreshing baths in the mornings and become sweaty and fragrant by nightfall. The day seemed to last much longer than its 24 hours, for there was no need to hurry - there was nowhere to go and nothing to buy.
